<a href="https://colab.research.google.com/github/Daniel-2003-python/IAyRedesN_ENE-JUN_2025/blob/main/Proyecto%20Final/PROYECTO_OFICIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Instalar librerías necesarias
!pip install opencv-python pyserial

# 3. Importar librerías
import cv2
import numpy as np
import tensorflow as tf
import serial
import time

# 4. Cargar el modelo desde Drive
modelo = tf.keras.models.load_model('/content/drive/MyDrive/proyectos/lenguaje_senas/modelo_senas.h5')
clases = ['A', 'C']  # ← reemplaza con tus clases reales

# 5. Configurar puerto serial (ajusta a tu puerto)
# EN COLAB NO FUNCIONA LA COMUNICACIÓN SERIAL DIRECTA.
# Este bloque es para uso local en tu PC, lo puedes copiar en un script .py de Python.
'''
puerto = serial.Serial('COM4', 9600)  # ← Cambia a tu puerto real
time.sleep(2)  # Espera a que el ARDUINO UNO se reinicie
'''

# 6. Iniciar cámara (funciona en PC local, no en Colab)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocesar imagen
    img = cv2.resize(frame, (64, 64))  # tamaño que usaste en el entrenamiento
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Predecir
    pred = modelo.predict(img)
    idx = np.argmax(pred)
    clase = clases[idx]
    confianza = pred[0][idx]

    # Mostrar resultado en la imagen
    cv2.putText(frame, f'Letra: {clase} ({confianza*100:.1f}%)', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Reconocimiento de Senas', frame)

    # ENVIAR SEÑAL AL ESP32 (localmente)
    '''
    if clase == 'A':
        puerto.write(b'1')  # abre compuerta 1
    elif clase == 'C':
        puerto.write(b'2')  # abre compuerta 2
    else:
        puerto.write(b'0')  # no hacer nada
    '''

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Finalizar
cap.release()
cv2.destroyAllWindows()
# puerto.close()  # solo si se usó serial

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


# Dataset Path / Route
dataset_path = '/content/drive/MyDrive/Robot Sensors/archive/data.csv'

# Parameters
img_height, img_width = 64, 64
batch_size = 32

# Image Generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Updated path to your image directory
# Ensure that 'archive' or the correct directory name is used and it exists
# If the 'archive' directory is within '/content/drive/MyDrive/Robot Sensors/'
# Then the path should be '/content/drive/MyDrive/Robot Sensors/archive'
train_gen = datagen.flow_from_directory(
    '/content/drive/MyDrive/Robot Sensors/archive', # Replace with the correct path to your image directory
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    '/content/drive/MyDrive/Robot Sensors/archive', # Replace with the correct path to your image directory
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save Model in Google Drive Archive
model.save('/content/drive/MyDrive/proyectos/lenguaje_senas/modelo_senas.h5')

Found 2101 images belonging to 2 classes.
Found 525 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 608s 9s/step - accuracy: 1.0000 - loss: 0.0482 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 409ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 26s 400ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 405ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 41s 405ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 406ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 26s 400ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 25s 375ms